# GOAL
Identify non professional TalentCards users in GBQ.

# PACKAGES

In [2]:
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
import yaml

# PARAMETERS

In [3]:
project_id = "analytics-dev-308300"

credentials = service_account.Credentials.from_service_account_file(
    "../keys/gcp_key.json",
)

# FUNCTIONS

# DATA WRANGLING

CSV file was manually generated:
- extract list of non-pro users from TalentCards (filter)
- manipulate the excel file to include the assotiation and clean any data besides identifier and association
- export as csv

In [4]:
df_non_pro=pd.read_csv('../params/non_professional_users.csv',sep=';')
df_non_pro['Identifier']=df_non_pro['Identifier'].replace({'-':''},regex=True)
df_non_pro

,Identifier,Association
0,AQGJEGFMZPFF,Humane
1,AQZWKMAQHJFL,Danone
2,AQCAGKPUKWUI,Humane
3,AQAISHKLLSMM,Humane
4,AQNCCPYYWMAN,Danone
5,AQSNUTGKIPXA,Danone
6,AQOJUJAYOCKL,Danone
7,AQBJIIYALBBE,Humane
8,AQHQBUDACZMX,Humane
9,AQQOKIWJESYZ,Danone


In [5]:
query = """
SELECT DISTINCT
    user_id, access_token
FROM
    dtm_engagement.dim_users
"""

df_bq_users=pd.read_gbq(query=query, credentials=credentials, project_id=project_id)

In [6]:
df_bq_users_flagged=(
    df_bq_users.merge(df_non_pro,left_on='access_token',right_on='Identifier')
    .drop(columns=['Identifier','access_token'])
    .rename(columns={'Association':'association'}))
df_bq_users_flagged

,user_id,association
0,20029,Humane
1,20029,Humane
2,20029,Humane
3,20030,Humane
4,20030,Humane
5,20032,Humane
6,20100,Humane
7,20118,Humane
8,20129,Humane
9,20136,Danone


In [7]:
df_bq_users_flagged.to_gbq('talentcards.non_pro_users',if_exists='replace')

1it [00:03,  3.19s/it]
